1. Install ElasticSearch (ES).
2. Install an ES plugin for Polish https://github.com/allegro/elasticsearch-analysis-morfologik 
3. Define an ES analyzer for Polish texts containing:
   1. standard tokenizer
   1. synonym filter with the following definitions:
      1. kpk - kodeks postępowania karnego
      1. kpc - kodeks postępowania cywilnego
      1. kk - kodeks karny
      1. kc - kodeks cywilny
   1. Morfologik-based lemmatizer
   1. lowercase filter

In [15]:
import re
import tarfile
import os
import matplotlib.pyplot as plt
import numpy as np
import regex
import pandas as pd
import morfeusz2
%matplotlib inline



from elasticsearch import Elasticsearch
from elasticsearch_dsl import analyzer, tokenizer

client = Elasticsearch("http://localhost:9200")
resp = client.info()
resp

{'name': 'x-dell',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'hrJwCHZtSGm_1kBHAztvag',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [110]:
print (es.VERSION)


AttributeError: 'Elasticsearch' object has no attribute 'VERSION'

In [7]:
merged_law = open("merged_law.txt", "w+b")


In [112]:
from elasticsearch_dsl import  *
from elasticsearch import *
from datetime import datetime

from elasticsearch_dsl import connections

connections.create_connection(hosts=['localhost'])
es = Elasticsearch()



my_analyzer = analyzer('morfologik',
    tokenizer=tokenizer('standard'),
    filter=["morfologik_stem"]
)

class Ustawa(Document):
    # title = Text()
    created_at = Date(default_timezone='UTC')
    content = Text()
    
    class Index:
        name = 'ustawa'

Ustawa.init()


path="../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as f: # open in readonly mode
        
        first = Ustawa(title='My First Blog Post, yay!',content=f.read())
        first.save()
        # break

# es.indices.create(index='test-index', ignore=400)




s = Search(using=es, index="ustawa").query("match_phrase", query='ustawa', analyzer='morfologik')

print(s.execute())



RequestError: RequestError(400, 'parsing_exception', "[match_phrase] query doesn't support multiple fields, found [query] and [analyzer]")

In [99]:
s = Search(using=es, index="ustawa").query("multi_match", query='ustawa', analyzer='morfologik')
# s = Search(using=es, index="ustawa").query(query='ustawa', analyzer='morfologik')

print()
for i in s.scan():
    print(i)

RequestError: RequestError(400, 'search_phase_execution_exception', '[multi_match] analyzer [morfologik] not found')

In [84]:
body = {
    # "from":0,
    # "size":1,
    "query": {
        "match_phrase": {
            "sentence":"Ustawa"
        }
    }
}

res = es.search(index="ustawa", body=body)

res

<ipython-input-84-23f7e0242f2c>:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index="ustawa", body=body)


{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [96]:
import os
path="../data/ustawy"
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as f: # open in readonly mode
        print(f.read())
        text=f.read()
        break
      # do your stuff






Dz.U. z 1998 r. Nr 117, poz. 759                                      
                                        
                                        
                                        
                                        
                                        
                                     USTAWA
                             z dnia 25 lipca 1998 r.
                                        
                       o zmianie ustawy o systemie oświaty
                                        
                                        
                                     Art. 1.
W ustawie z dnia 7 września 1991 r. o systemie oświaty (Dz.U. z 1996 r. Nr 67,
poz. 329, Nr 106, poz. 496 oraz z 1997 r. Nr 28, poz. 153 i Nr 141, poz. 943)
wprowadza się następujące zmiany:
1) w art. 1:
   a) po pkt 5 dodaje się pkt 5a w brzmieniu:
      "5a) opiekę nad uczniami ze znacznymi lub sprzężonymi dysfunkcjami
         poprzez umożliwianie realizowania indywidualnych form i programó

In [35]:
from datetime import datetime
from elasticsearch_dsl import Document, Date, Nested, Boolean, \
    analyzer, InnerDoc, Completion, Keyword, Text

html_strip = analyzer('html_strip',
    tokenizer="standard",
    filter=["standard", "lowercase", "stop", "snowball"],
    char_filter=["html_strip"]
)

class Comment(InnerDoc):
    author = Text(fields={'raw': Keyword()})
    content = Text(analyzer='snowball')
    created_at = Date()

    def age(self):
        return datetime.now() - self.created_at

class Post(Document):
    title = Text()
    title_suggest = Completion()
    created_at = Date(default_timezone='UTC')
    published = Boolean()
    category = Text(
        analyzer=html_strip,
        fields={'raw': Keyword()}
    )

    comments = Nested(Comment)

    class Index:
        name = 'blog'

    def add_comment(self, author, content):
        self.comments.append(
          Comment(author=author, content=content, created_at=datetime.now()))

    def save(self, ** kwargs):
        self.created_at = datetime.now()
        return super().save(** kwargs)

# Post.init()

# instantiate the document
first = Post(title='My First Blog Post, yay!', published=True)
# assign some field values, can be values or lists of values
first.category = ['everything', 'nothing']
# every document has an id in meta
first.meta.id = 47


# save the document into the cluster
first.save()


KeyError: "There is no connection with alias 'default'."

In [4]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch()

doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", id=1, document=doc)
print(res['result'])

res = es.get(index="test-index", id=1)
print(res['_source'])

es.indices.refresh(index="test-index")

res = es.search(index="test-index", query={"match_all": {}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])



created
{'author': 'kimchy', 'text': 'Elasticsearch: cool. bonsai cool.', 'timestamp': '2021-10-30T22:11:21.389803'}
Got 1 Hits:
2021-10-30T22:11:21.389803 kimchy: Elasticsearch: cool. bonsai cool.


In [2]:
from elasticsearch_dsl.query import MultiMatch, Match

# {"multi_match": {"query": "python django", "fields": ["title", "body"]}}
MultiMatch(query='python django', fields=['title', 'body'])

# {"match": {"title": {"query": "web framework", "type": "phrase"}}}
Match(title={"query": "web framework", "type": "phrase"})


Match(title={'query': 'web framework', 'type': 'phrase'})

4. Define an ES index for storing the contents of the legislative acts.
5. Load the data to the ES index.
6. Determine the number of legislative acts containing the word **ustawa** (in any form).

7. Determine the number of occurrences of the word **ustawa** by searching for this particular form, including the other inflectional forms.
8. Determine the number of occurrences of the word **ustaw** by searching for this particular form, including the other inflectional forms.
9. Determine the number of legislative acts containing the words **kodeks postępowania cywilnego** 
   in the specified order, but in any inflection form.
10. Determine the number of legislative acts containing the words **wchodzi w życie** 
   (in any form) allowing for up to 2 additional words in the searched phrase.
11. Determine the 10 documents that are the most relevant for the phrase **konstytucja**.
12. Print the excerpts containing the word **konstytucja** (up to three excerpts per document) 
   from the previous task.